## Operacionalização do Modelo de Risco Proativo (MLOps)

#### Do Código à Produção: Estrutura da Intervenção Preditiva

Este notebook detalha o pipeline de MLOps para implantação do modelo LightGBM em produção, focando na automação da Feature Engineering, na aplicação do **Threshold de Negócio (0.20)** e na integração com os canais de atendimento.

---

### 1. FLUXO DO PIPELINE DE PRODUÇÃO

O modelo é projetado para operar em lote (batch) ou tempo real (real-time), dependendo da criticidade. O fluxo principal é o seguinte:

- Ingestão de Dados (Data Source): Recebe os dados brutos do cliente (transações, limite, SCR, etc.)

- Pré-Processamento (Feature Store): Executa o pipeline de Feature Engineering (cálculo das janelas de 3 meses, garantindo a recriação fiel das features do modelo).

- Inferência (Model Server): O modelo LightGBM gera a probabilidade de risco ($\text{y\_proba}$).

- Decisão de Negócio: Aplica o Threshold Otimizado (0.20) e encaminha a decisão ao Canal de Intervenção.2.2. Código de Demonstração (Simulação de Inferência)O código de simulação de inferência em produção deve ser o mais limpo possível, focando na aplicação do threshold.Código de Demonstração (Python):


In [ ]:
# =============================================================================
# 4. SIMULAÇÃO DA INFERÊNCIA EM PRODUÇÃO (Model Server)
# =============================================================================

import pickle
import numpy as np
import pandas as pd

# Carregar o modelo treinado
MODEL_FILE = 'lgbm_credit_model_final.pkl'
with open(MODEL_FILE, 'rb') as f:
    lgbm_prod = pickle.load(f)

# THRESHOLD OTIMIZADO
THRESHOLD_OTIMIZADO = 0.20

# -----------------------------------------------------------------------------
# Exemplo de Payload (Novos Dados do Cliente X no Mês 18)
# O Feature Engineering precisa ser executado exatamente como no Treinamento
# -----------------------------------------------------------------------------

# Criar um DataFrame de Teste (1 cliente) com as 15 features finais do Treinamento
# (Apenas as features numéricas e os resultados do One-Hot Encoding das categóricas)
X_single_client = pd.DataFrame({
    'limite_cartao': [25000.0], 
    'scr_tcr_implicita': [0.15], # Sinal de Risco: Alto TCRI (comprometimento)
    'gasto_crescim_3m': [0.85],  # Sinal de Risco: Crescimento de 85% no gasto
    'scr_crescim_divida_3m': [0.60], # Sinal de Risco: Crescimento de 60% na dívida SCR
    'utilizacao_limite_media_3m': [0.88], # Sinal de Risco: Alto uso do limite
    'contagem_rotativo_3m': [0.0],
    
    # Exemplo das features categóricas (todas as 15 features devem estar aqui)
    'idade_faixa_26-40': [1], 'idade_faixa_41-65': [0], 'idade_faixa_65+': [0],
    'classe_social_C': [0], 'classe_social_D/E': [0],
    'regiao_Nordeste': [0], 'regiao_Norte': [0], 'regiao_Sudeste': [1], 'regiao_Sul': [0]
})

# 1. Gerar a Probabilidade (y_proba)
y_proba_pred = lgbm_prod.predict_proba(X_single_client)[:, 1][0]

# 2. Aplicar a Decisão de Negócio
decisao = 'ALERTA (Intervenção Imediata)' if y_proba_pred >= THRESHOLD_OTIMIZADO else 'NORMAL (Monitoramento)'

# -----------------------------------------------------------------------------
# Output da Inferência
# -----------------------------------------------------------------------------
print("="*50)
print(f"ANÁLISE DE RISCO PREDITIVO (Produção)")
print("="*50)
print(f"Probabilidade de Rotativo (y_proba): {y_proba_pred:.4f}")
print(f"Threshold de Decisão de Negócio: {THRESHOLD_OTIMIZADO:.2f}")
print("-" * 50)
print(f"STATUS DA DECISÃO: {decisao}")
print("-" * 50)
print("Encaminhar cliente para o canal de Intervenção Consultiva.")

##  Resultados do Modelo Preditivo - Validação Out-of-Time (OOT)

###  Performance do Modelo LightGBM

**Configuração do Modelo:**
- **Threshold de Decisão:** 0.20
- **Peso da Classe 1:** 4.19x
- **Amostras de Treino:** 170,000
- **Amostras de Teste (OOT):** 60,000

###  Métricas de Performance Validadas

| Métrica | Valor | Interpretação |
| :--- | :---: | :--- |
| **AUC Score** | **0.7776** |  **Alta capacidade preditiva** - acima do benchmark de 0.75 |
| **Recall (Classe 1)** | **100%** |  **Proteção total** - captura TODOS os clientes em risco de rotativo |
| **Precision (Classe 1)** | **36%** |  **Estratégico** - 64% de FPs convertidos em oportunidades |
| **Acurácia** | **55%** |  **Esperado** - resultado do trade-off Recall vs Precision |
| **F1-Score (Classe 1)** | **53%** |  **Balanceado** - considerando o desbalanceamento das classes |

###  Distribuição do Dataset

**Base Final de ML:**
- **Total de registros:** 230,000 observações
- **Classe 0 (Não Rotativo):** 79.11%
- **Classe 1 (Rotativo):** 20.89%

**Divisão Treino/Teste:**
- **Treino (Meses 1-17):** 170,000 amostras
- **Teste OOT (Meses 18-24):** 60,000 amostras

###  Matriz de Confusão - Análise Detalhada

|           | Previsto 0 | Previsto 1 | Total    |
|-----------|------------|------------|----------|
| **Real 0** | 17,919     | 26,808     | 44,727   |
| **Real 1** | 0          | 15,273     | 15,273   |
| **Total**  | 17,919     | 42,081     | 60,000   |

**Análise dos Resultados:**
-  **Verdadeiros Positivos (VP):** 15,273 clientes (risco real mitigado)
-  **Falsos Positivos (FP):** 26,808 clientes (oportunidades de fidelização)
-  **Verdadeiros Negativos (VN):** 17,919 clientes (corretamente identificados)
-  **Falsos Negativos (FN):** 0 clientes (NENHUM risco deixou de ser capturado)


**Análise dos Resultados:**
-  **Verdadeiros Positivos (VP):** 15,273 clientes (risco real mitigado)
-  **Falsos Positivos (FP):** 26,808 clientes (oportunidades de fidelização)
-  **Verdadeiros Negativos (VN):** 17,919 clientes (corretamente identificados)
-  **Falsos Negativos (FN):** 0 clientes (NENHUM risco deixou de ser capturado)

###  Top 10 Features por Importância (Gain)

| Posição | Feature | Importância | Categoria |
| :---: | :--- | :---: | :--- |
| **1** | `limite_cartao` | **354** |  Exposição ao Crédito |
| **2** | `scr_tcr_implicita` | **333** |  Risco Sistêmico |
| **3** | `utilizacao_limite_media_3m` | **297** |  Estresse de Liquidez |
| **4** | `scr_crescim_divida_3m` | **208** |  Deterioração Financeira |
| **5** | `gasto_crescim_3m` | **186** |  Estresse de Consumo |
| **6** | `contagem_rotativo_3m` | **40** |  Comportamento Rotativo |
| **7** | `regiao_Sudeste` | **26** |  Contexto Regional |
| **8** | `idade_faixa_26-40` | **22** |  Perfil Demográfico |
| **9** | `regiao_Sul` | **14** |  Contexto Regional |
| **10** | `idade_faixa_65+` | **13** |  Perfil Demográfico |

###  Insights Estratégicos

**Pontos Fortes:**
- **Recall 100%** garante proteção total contra perdas por rotativo
- **AUC 0.7776** comprova robustez preditiva do modelo
- **Top features** alinhadas com teoria de risco comportamental
- **Nenhum falso negativo** - risco totalmente mitigado

**Oportunidades de Otimização:**
- **26,808 FPs** representam base para estratégias de fidelização
- **Precision 36%** pode ser melhorada com refinamento contínuo
- **Features demográficas** com baixa importância - possível simplificação

###  Próximos Passos Imediatos

1. **Implementar** modelo em produção com threshold 0.20
2. **Desenvolver** estratégia de fidelização para os 26,808 FPs
3. **Monitorar** estabilidade das top 5 features
4. **Estabelecer** processo de retreinamento trimestral

---

*"Modelo validado com sucesso: 100% de proteção contra risco de rotativo com AUC robusto de 0.7776"*
